In [66]:
import torch
import numpy as np
print(f'PyTorch version: {torch.__version__}')
print(f'Is MPS available? {torch.backends.mps.is_available()}')
print(f'Does MPS exists? {torch.backends.mps.is_built()}')

PyTorch version: 2.6.0
Is MPS available? True
Does MPS exists? True


# Introducción

*Las redes neuronales son modelos estadísticos que nos permiten resolver problemas complejos gracias a su capacidad de aprender patrones en los datos. Por ejemplo, podemos utilizar estos modelos para detectar tumores en imágenes médicas o transformar texto en audio.*

*La manera que tenemos de entrenar estos modelos es con ejemplos. Por ejemplo, si quisiéramos entrenar un modelo que distinga entre perros y gatos, deberíamos mostrarle imágenes con perros y gatos, comparar las predicciones del modelo con las etiquetas reales (i.e., perro o gato), y luego ajustar el modelo para que sus predicciones se vuelvan más precisas. El problema que tenemos es que las computadoras, a diferencia de los humanos, solo entienden de números. Entonces, para poder entrenar estos modelos, primero necesitamos transformar nuestros datos (i.e., imágenes) en un formato que la computadora pueda entender.*

*PyTorch nos proporciona una estructura de datos, los **tensores**, que nos permiten representar nuestros datos en números. A diferencia de su significado en matemática, los tensores en PyTorch se refieren a arrays multidimensionales, similares a los arrays de NumPy, pero con capacidades adicionales que los hacen especialmente útiles para entrenar modelos de Deep Learning*.

<img src="attachments/tensors-in-pytorch.png" width="900" height="700" align="center" />

## Crear tensores

### Escalares

In [67]:
# Create a scalar
scalar = torch.tensor(7.0)
scalar

tensor(7.)

*Para extraer el número del tensor podemos utilizar el método `item()`.*

In [68]:
# Get a tensor back as an integer or float
print(f'Escalar: {scalar.item()}. Type: {type(scalar.item())}')

Escalar: 7.0. Type: <class 'float'>


### Vectores

*Los vectores son tensores **unidimensionales**. Utilizando el atributo `ndim` podemos validar que el tensor que creamos tiene una única dimensión. Podemos mirar con más detalle las dimensiones del tensor con el atributo `shape` (notar que su dimensión tiene tamaño dos porque tiene solo dos elementos).*

In [69]:
# Create a vector
vector = torch.tensor([7, 7])
vector

tensor([7, 7])

In [70]:
# Dimensions
print(f'Number of dimensions: {vector.ndim}')

Number of dimensions: 1


In [71]:
print(f'Shape: {vector.shape}')

Shape: torch.Size([2])


### Matrices

*Las matrices son tensores **bi-dimensionales**, en donde la primera dimensión representa las filas y la segunda dimensión representa las columnas. El tamaño que tienen las dimensiones hacen referencia la cantidad de filas y la cantidad de columnas de la matriz.*

In [72]:
MATRIX = torch.tensor([[7, 8], [8, 9]])
MATRIX

tensor([[7, 8],
        [8, 9]])

In [73]:
print(f'Number of dimensions: {MATRIX.ndim}')

Number of dimensions: 2


In [74]:
print(f'Shape: {MATRIX.shape}')

Shape: torch.Size([2, 2])


### Tensores

*Si bien los vectores y las matrices son, al menos en PyTorch, tensores de 1D y 2D, respectivamente, llamamos tensor a los arrays **n-dimensionales** (con $n > 0$). Por ejemplo, en el código debajo tenemos un tensor de tres dimensiones.*

In [75]:
# Create a tensor
TENSOR = torch.tensor([[[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]]])
TENSOR

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [76]:
print(f'Number of dimensions: {TENSOR.ndim}')

Number of dimensions: 3


In [77]:
print(f'Shape: {TENSOR.shape}')

Shape: torch.Size([1, 3, 3])


*Para crear un tensor a partir de otro(s) tensor(es) como input(s), es necesario utilizar funciones como `torch.stack()` o `torch.vstack()`. La clase `torch.tensor()` solo acepta objetos nativos de Python. En este caso utilizamos la función `torch.stack()` que nos permite concatenar tensores sobre una nueva dimensión.*

In [78]:
MATRIX_1 = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
MATRIX_2 = torch.tensor([[10, 11, 12], [13, 14, 15], [16, 17, 18]])

# The torch.tensor() class expects Python native data types, so in order to create a tensor from two tensors
TENSOR = torch.stack([MATRIX_1, MATRIX_2])
TENSOR

tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[10, 11, 12],
         [13, 14, 15],
         [16, 17, 18]]])

In [79]:
print(f'Number of dimensions: {TENSOR.ndim}')

Number of dimensions: 3


In [80]:
print(f'Shape: {TENSOR.shape}')

Shape: torch.Size([2, 3, 3])


## Tensores en memoria

*A diferencia de las estructuras de datos nativas de Python, como las listas, los tensores se guardan en memoria como un bloque contiguo. Qué quiere decir esto? Que a diferencia de las listas, donde cada elemento tiene su propio espacio en memoria, los tensores se encuentran guardados en un único espacio en memoria, donde cada elemento se encuentra uno al lado del otro (no importa cuantas dimensiones tenga nuestro tensor, en memoria siempre se va a guardar como un vector unidimensional). Esto hace que sea mucho más rápido acceder secuencialmente a los elementos de un tensor y que sea mucho más sencillo vectorizar operaciones con ellos.*

*La siguiente imagen es muy descriptiva de como se guardan las listas (u otros objetos nativos de Python) y los tensores en memoria:*
<img src="attachments/tensors-in-memory.png" width="900" height="700" align="center" />

### Metadata

*Como ya dijimos, los tensores se guardan de manera contigua en memoria. Para poder acceder a los elementos utilizando los índices, un tensor tiene información adicional que permite definirlos inequívocamente:*
- *`size`. Nos dice cuantos elementos tiene el tensor en cada dimensión.*
- *`stride`. Nos dice cuantos elementos en memoria tenemos que movernos para obtener el siguiente elemento de una dimensión. Por ejemplo, en la imagen de abajo podemos ver que para obtener el siguiente elemento en la segunda dimensión (columnas) tenemos que movernos un lugar en memoria, y para obtener el siguiente elemento en la primera dimensión (filas) tenemos que movernos tres lugares en memoria.*
- *`offset`. Nos dice cuál es el primer elemento del tensor en la memoria.*
- *`storage`. Nos dice donde se encuentra guardado el tensor en memoria.*

<img src="attachments/tensor-metadata.png" width="900" height="700" align="center" />

## Tensores aleatorios

*Por qué queremos aprender a generar tensores de forma aleatoria? Porque los pesos de las redes neuronales se inicializan es esta manera. Entonces, al comenzar el entrenamiento de nuestra red neuronal debemos pasarle un tensor que represente los pesos iniciales, el cual tenemos que poder generar de manera aleatoria.*

In [81]:
# Tensor de dos dimensiones de tamaño (3, 4)
random_tensor = torch.rand(size=(2, 3))
random_tensor

tensor([[0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346]])

In [82]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Tamaños: {random_tensor.shape}')

Dimensions: 2
Tamaños: torch.Size([2, 3])


In [83]:
random_tensor = torch.rand(size=(10, 2, 3))
random_tensor

tensor([[[0.5936, 0.8694, 0.5677],
         [0.7411, 0.4294, 0.8854]],

        [[0.5739, 0.2666, 0.6274],
         [0.2696, 0.4414, 0.2969]],

        [[0.8317, 0.1053, 0.2695],
         [0.3588, 0.1994, 0.5472]],

        [[0.0062, 0.9516, 0.0753],
         [0.8860, 0.5832, 0.3376]],

        [[0.8090, 0.5779, 0.9040],
         [0.5547, 0.3423, 0.6343]],

        [[0.3644, 0.7104, 0.9464],
         [0.7890, 0.2814, 0.7886]],

        [[0.5895, 0.7539, 0.1952],
         [0.0050, 0.3068, 0.1165]],

        [[0.9103, 0.6440, 0.7071],
         [0.6581, 0.4913, 0.8913]],

        [[0.1447, 0.5315, 0.1587],
         [0.6542, 0.3278, 0.6532]],

        [[0.3958, 0.9147, 0.2036],
         [0.2018, 0.2018, 0.9497]]])

In [84]:
print(f'Dimensions: {random_tensor.ndim}')
print(f'Tamaños: {random_tensor.shape}')

Dimensions: 3
Tamaños: torch.Size([10, 2, 3])


*Las imagenes se pueden representar como tensores de tres dimensiones, en donde la primer dimensión representa los canales de colores (RGB) y la segunda y tercer dimensión representa la cantidad de píxeles de la imágen. Por ejemplo, si tenemos una imagen a color de 1024x1024 y la convertimos en un tensor, entonces tendríamos un tensor de tres dimensiones con tamaño `torch.Size([3, 1024, 1024])`.*

In [85]:
random_image_tensor = torch.rand(size=(3, 1024, 1024))
random_image_tensor

tensor([[[0.6666, 0.9811, 0.0874,  ..., 0.0707, 0.6793, 0.9227],
         [0.5303, 0.1988, 0.9099,  ..., 0.8482, 0.8240, 0.6414],
         [0.6738, 0.6613, 0.5519,  ..., 0.1718, 0.1639, 0.0116],
         ...,
         [0.2805, 0.7836, 0.6706,  ..., 0.5299, 0.3607, 0.4438],
         [0.6341, 0.8021, 0.2384,  ..., 0.7361, 0.5131, 0.0082],
         [0.2442, 0.3589, 0.8952,  ..., 0.6155, 0.4215, 0.6004]],

        [[0.2620, 0.3809, 0.0809,  ..., 0.1393, 0.5335, 0.1217],
         [0.7090, 0.4542, 0.6700,  ..., 0.2559, 0.0658, 0.7274],
         [0.7688, 0.7784, 0.5482,  ..., 0.1722, 0.1141, 0.1717],
         ...,
         [0.5084, 0.3411, 0.4173,  ..., 0.2874, 0.1223, 0.0622],
         [0.5953, 0.5328, 0.2545,  ..., 0.2621, 0.1383, 0.2893],
         [0.7187, 0.6074, 0.9588,  ..., 0.7932, 0.8076, 0.1934]],

        [[0.2462, 0.8442, 0.7218,  ..., 0.8065, 0.4769, 0.2840],
         [0.3523, 0.1319, 0.9459,  ..., 0.8241, 0.6992, 0.9141],
         [0.9984, 0.4112, 0.9072,  ..., 0.2477, 0.0688, 0.

In [86]:
print(f'Dimensiones: {random_image_tensor.ndim}')
print(f'Shape: {random_image_tensor.shape}')

Dimensiones: 3
Shape: torch.Size([3, 1024, 1024])


*Siempre que trabajemos con procesos aleatorios (o pseudo-aleatorios), vamos a observar diferentes resultados en las ejecuciones. Incluso teniendo la misma semilla, puede que el mismo proceso aleatorio genere distintos resultados si lo ejecutamos en distintos dispositivos (i.e., CPU o GPU). Para más información acerca de como trabajar con procesos no-determinísticos en PyTorch, leer la siguiente [documentación](https://pytorch.org/docs/stable/notes/randomness.html).*

*Podemos definir una semilla que nos permita reproducir los resultados de un experimento utilizando la función `torch.manual_seed()` (se define la misma semilla para todos los dispositivos). Ahora, ejecutar multiples veces el mismo código nos devuelve el mismo resultado.*

In [87]:
torch.manual_seed(42)
random_tensor_1 = torch.rand(size=(1, 5))
random_tensor_2 = torch.rand(size=(1, 5))

print(f'Random tensor 1: {random_tensor_1}')
print(f'Random tensor 2: {random_tensor_2}')

# We can compare two tensors using the function torch.equal() or the operator ==
print(f'Are the tensors equal? {torch.equal(random_tensor_1, random_tensor_2)}')

Random tensor 1: tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904]])
Random tensor 2: tensor([[0.6009, 0.2566, 0.7936, 0.9408, 0.1332]])
Are the tensors equal? False


*Notar que, a pesar de estar utilizando la misma semilla, los tensores que generamos no son iguales. Para obtener dos tensores iguales deberíamos a ejecutar `torch.manual_seed()` antes de crear cada uno de los tensores.*

In [88]:
torch.manual_seed(42) # Definimos la semilla para el primer tensor
random_tensor_1 = torch.rand(size=(1, 5))
torch.manual_seed(42) # Definimos la semilla para el segundo tensor
random_tensor_2 = torch.rand(size=(1, 5))

print(f'Random tensor 1: {random_tensor_1}')
print(f'Random tensor 2: {random_tensor_2}')

# We can compare two tensors using the function torch.equal() or the operator ==
print(f'Are the tensors equal? {torch.equal(random_tensor_1, random_tensor_2)}')

Random tensor 1: tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904]])
Random tensor 2: tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904]])
Are the tensors equal? True


## Tensor-like objects

In [89]:
range_tensor = torch.arange(start=0, end=1000, step=36)
range_tensor

tensor([  0,  36,  72, 108, 144, 180, 216, 252, 288, 324, 360, 396, 432, 468,
        504, 540, 576, 612, 648, 684, 720, 756, 792, 828, 864, 900, 936, 972])

In [90]:
print(f'Dimension: {range_tensor.ndim}')
print(f'Shape: {range_tensor.shape}')
print(f'Dtype: {range_tensor.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


*Cuando decimos que vamos a crear o que creamos un objeto que es "tensor-like", nos referimos a crear un nuevo tensor que tenga la misma dimensión, tamaño y tipo de dato que otro tensor ya existente, pero con otros valores.*

*Por ejemplo, podemos creamos un nuevo tensor a partir del tensor `range_tensor` que tenga la misma dimensión, tamaño y tipo de dato, pero que la información que contiene sea distinta (solo tenga ceros).*

In [91]:
zeros = torch.zeros_like(input=range_tensor)
zeros

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [92]:
print(f'Dimension: {zeros.ndim}')
print(f'Shape: {zeros.shape}')
print(f'Dtype: {zeros.dtype}')

Dimension: 1
Shape: torch.Size([28])
Dtype: torch.int64


## Atributos de los tensores

### Tipo de dato


*Por default, PyTorch crea los tensores utilizando el tipo de dato `torch.float32`. Sin embargo, hay muchos más tipos de datos [disponibles](https://pytorch.org/docs/stable/tensors.html#data-types), los cuales se diferencian principalmente por la precisión númerica que nos aportan. Por ejemplo, el tipo de dato `torch.float64` tiene mucho más precisión numérica que el tipo de dato `torch.float32` (casi el doble de decimales), pero consume mucho más memoria.*

*Podemos definir el tipo de dato que queremos para nuestro tensor utilizando el argumento `dtype`.*

In [93]:
TENSOR_0 = torch.tensor([3., 2., 1.], dtype=torch.float64)
TENSOR_0

tensor([3., 2., 1.], dtype=torch.float64)

In [94]:
TENSOR_0.dtype

torch.float64

*Podemos modificar el tipo de dato utilizando el método `torch.Tensor.type()` o el método `torch.Tensor.to()`. Este último método es el más recomendado, ya que también nos permite modificar otros atributos del tensor.*

In [95]:
# Change tensor's datatype
TENSOR_1 = TENSOR_0.to(dtype=torch.float16) # or use torch.Tensor.to()
TENSOR_1

tensor([3., 2., 1.], dtype=torch.float16)

*Cuando realizamos operaciones matemáticas entre dos tensores que tienen tipos de datos distintos, PyTorch utiliza una **regla de promoción** para definir el tipo de dato del tensor resultante (tendrá el tipo de dato con mayor precisión). Por ejemplo, si multiplicamos un tensor con tipo de dato `torch.float32` y otro con tipo de dato `torch.float64`, el tensor que obtenemos después de la operación es `torch.float64`.*

*Esto es algo importante a tener en cuenta porque:*
1. *Podemos llegar a consumir más memoria de la esperada si las operaciones que realizamos están constantemente incrementando la precisión númerica de nuestros tensores.*
2. *Las operaciones entre tensores con distintos tipos de datos suelen ser menos performantes porque PyTorch también se tiene que encargar de convertir los datos.*
3. *Algunos dispositivos tienen soporte limitado para algunos tipos de datos (e.g., las GPU no soportan tensores con tipos de datos `torch.float64` o las `mps` no soportan los tensores con tipos de datos `torch.float16`).*

In [96]:
TENSOR_0 * TENSOR_1

tensor([9., 4., 1.], dtype=torch.float64)

### Dispositivo

*Podemos también definir el **dispositivo** en donde se guarda y ejecuta el tensor. Hay varios tipos de dispositivos que podemos utilizar, pero los más utilizados son:*
1. *`cpu`: nos permite guardar y ejecutar los tensores en la memoria RAM de nuestra computadora o máquina virtual.*
2. *`cuda`: nos permite guardar y ejecutar los tensores en la GPU. Si tenemos disponibles más de una GPU, podemos especificar cual queremos utilizar, e.g.,  `cuda:0` (primer GPU), `cuda:1` (segunda GPU), `cuda:2` (tercera GPU), etc.*
3. *`mps`: nos prmite utilizar la GPU integrada en las computadoras con chips Apple Silicon (similar a como `cuda` nos permite utilizar las GPUs de NVIDIA).*

*Por qué es importante definir el dispositivo?:*
1. *Si tenemos dos tensores que están guardados en dos dispositivos distintos no podemos realizar operaciones entre ellos.*
2. *Si queremos guardar y ejecutar los tensores en la GPU, tenemos que seleccionar el dispositivo `cuda`.*
3. *Mover tensores de un dispositivo a otro puede ser computacionalmente costoso.*

In [97]:
TENSOR.device

device(type='cpu')

*Podemos elegir el dispositivo en donde queremos guardar y ejecutar el tensor al momento de crearlo:*

In [98]:
# Define a torch.device
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a new tensor with specified device
TENSOR_2 = torch.rand(size=(3, 1024, 1024), device='cpu')
print(f'Tensor: {TENSOR_2}')
print(f'Device: {TENSOR_2.device}')

Tensor: tensor([[[0.6009, 0.2566, 0.7936,  ..., 0.7124, 0.2065, 0.5760],
         [0.1976, 0.7499, 0.2813,  ..., 0.3988, 0.7365, 0.6829],
         [0.0499, 0.2046, 0.5168,  ..., 0.3662, 0.0104, 0.4482],
         ...,
         [0.8996, 0.0490, 0.7916,  ..., 0.8479, 0.9953, 0.1614],
         [0.0894, 0.1535, 0.1571,  ..., 0.3873, 0.8991, 0.6478],
         [0.0201, 0.3778, 0.1633,  ..., 0.9565, 0.9679, 0.9763]],

        [[0.8273, 0.4332, 0.7660,  ..., 0.5370, 0.0109, 0.8638],
         [0.0710, 0.9027, 0.1023,  ..., 0.8887, 0.2716, 0.5103],
         [0.0258, 0.8463, 0.4397,  ..., 0.2049, 0.7828, 0.2040],
         ...,
         [0.0307, 0.5348, 0.4726,  ..., 0.1216, 0.4119, 0.9457],
         [0.0489, 0.9974, 0.1015,  ..., 0.6457, 0.5405, 0.1967],
         [0.7970, 0.3450, 0.1873,  ..., 0.0286, 0.7181, 0.1854]],

        [[0.5284, 0.8363, 0.7645,  ..., 0.4472, 0.3017, 0.6413],
         [0.6611, 0.7715, 0.3460,  ..., 0.7348, 0.1475, 0.1933],
         [0.4940, 0.4051, 0.1993,  ..., 0.4237, 0.

*Podemos también utilizar la función `torch.Tensor.to` para cambiar el tipo de dato o el dispositivo de un tensor (notar que es necesario re-asignar el tensor a la variable):*

In [99]:
TENSOR_2 = TENSOR_2.to(device='mps')
print(f'Device: {TENSOR_2.device}')

Device: mps:0


### Gradientes

*Otro argumento importante de los tensores es el argumento `requires_grad`. Cuando definimos `requires_grad=True` en un tensor, le estamos pidiendo a PyTorch que guarde todas las operaciones realizadas sobre ese tensor, para así poder calcular los gradientes. Esto es muy útil cuando queremos entrenar un modelo de Deep Learning, ya que nos permite calcular los gradientes de la función de pérdida con respecto a los pesos del modelo utilizando backpropagation.*

*Notar que en el siguiente ejemplo creamos una nueva variable, `y`, que es igual a la suma de los elementos del tensor `X`. Luego, para calcular los gradientes de `y` con respecto a los elementos en `X`, llamamos a la función `.backward()` desde la variable `y`. Sin embargo, los gradientes los podemos encontrar en el tensor `X` (eso tiene sentido porque los gradientes nos dicen como cambia `y` cuando cambia un elemento de `X`.*

In [100]:
# Creates a tensor and operates on it
X = torch.tensor([3., 2., 1.], requires_grad=True, device='mps')

# Sum all elements of the tensor and compute the gradients
y = X.sum()
y.backward()
print(f'Gradients after automatic differentiation: {TENSOR_0.grad}')

Gradients after automatic differentiation: None


## PyTorch y NumPy

*Podemos crear tensores a partir de arrays de NumPy, y viceversa. Esto es muy útil porque podemos utilizar las funcionalidades NumPy para manipular nuestros datos, y luego utilizar los tensores de PyTorch para entrenar nuestros modelos de Deep Learning.*

*Podemos crear un tensor a partir de un array de NumPy utilizando la función `torch.from_numpy(ndarray)`. Si queremos convertir un tensor a un array de NumPy, tenemos que utilizar el método `torch.Tensor.numpy()`.*

In [101]:
array = np.arange(1., 10.).reshape((3, 3))
print(f'Array de NumPy: \n{array}')

Array de NumPy: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


In [102]:
tensor = torch.from_numpy(array)
print(f'Tensor de PyTorch: \n{tensor}')

Tensor de PyTorch: 
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]], dtype=torch.float64)


*Algo muy importante a tener en cuenta es que el tipo de dato predeterminado por NumPy es diferente al de PyTorch (NumPy utiliza `float64` mientras que PyTorch utiliza `float32`). El tensor que creamos a partir del array de NumPy hereda el tipo de dato del array, lo cual podría generar problemas al querer operar entre tensores (si los tipos de datos no coinciden).*

In [103]:
print(f"Array's datatype: {array.dtype}")
print(f"Tensor's datatype: {tensor.dtype}")

Array's datatype: float64
Tensor's datatype: torch.float64


In [104]:
tensor = tensor.type(torch.float32)
print(f"Tensor's datatype: {tensor.dtype}")

Tensor's datatype: torch.float32
